# Draft analysis 

---

Group name: Gruppe A | Ardit Fazliu, Lukas Laib, Marcus Wild

---


## Introduction
*This section includes an introduction to the project motivation, data and research question. Include a data dictionary.*

### Subject

Der Vertrieb und Verkauf von Waren über das Internet ist zu einem wichtigen Vertriebskanal der modernen, digitalisierten Wirtschaft geworden. Diese Entwicklung wurde durch Covid19-Pandemie weiter verstärkt. So stiegen in Deutschland die im B2C E-Commerce erzielten Umsätze im Jahr 2020 gegenüber dem Vorjahr um fast 23% auf knapp 73 Milliarden Euro (Statista, 2022a). Auch innerhalb des B2B E-Commerce wird in den kommenden Jahren ein enormes Umsatzwachstum erwartet, in den USA wird der Umsatz im Jahr 2025 auf 2,5 Billionen steigen, dies entspricht einer Wachstumsrate von rund 11% p.a. (Statista, 2022b). 

### Motivation

Während die Umsätze im E-Commerce Jahr für Jahr steigen, sind die Margen im Onlinehandel für einen Großteil der Händler marginal. Dennoch müssen Unternehmen hohe Investitionen in das Online-Geschäft tätigen, um Umsätze im stark umkämpften Online-Geschäft zu erzielen (Alvarez & Marsal, 2021). Damit sich diese nachhaltig für das Unternehmen lohnen, müssen langfristig verlässliche Gewinne erwirtschaftet werden. Dieses Ziel kann nur erreicht werden, wenn externe Risiken für das Unternehmen minimiert und intern strategisch richtige Entscheidungen zur Portfolioerweiterung getroffen werden. Im folgenden soll ein Datensatz, bestehend aus Transaktionsdaten eines E-Commerce Unternehmen aus dem Vereinigten Königreich analysiert werden, um Abhängigkeiten von bestimmten  1.) Ländern oder Regionen 2) Kunden 3) Produkten bzw. Produktgruppen zu identifizieren und 4) Potenziale für Umsatzwachstum / Gewinnsteigerung zu entdecken.

### General Research Question

**Wie können Abhängigkeiten für den E-Commerce Shop langfristig reduziert werden und welche Maßnahmen können zur langfristigen Umsatzmaximierung getroffen werden?**

### Data Description

Der Datensatz zeigt Transaktionsdaten eines E-Commerce Unternehmen aus dem Vereinigten Koenigreich. Die Daten wurden zwischen dem 01.12.2010 und dem 09.12.2011 erfasst.

Jeder Beobachtungswert stellt innerhalb des Datensatzes eine Transaktion dar, wobei mehreren Transaktionen eine Rechnung(snummer) zugeteilt werden kann.

Der Datensatz enthaelt in der ursprünglichen Versionierung 541909 Beobachtungswerte und 8 Spalten.

### Data Dictionary
*Das Skalenniveau ist detaillierter angegeben. Statt "Numeric" wurde die Unterteilung zwischen intervall- und verhaeltnisskaliert verwendet.

| Spaltenname  | Beschreibung  | Skalenniveau | Format |
|---|---|---|---|
|InvoiceNo | Rechnungsnummer, 6-stellige Nummer, welche jeder Transaktion eindeutig zuordnet. <br>Falls die Nummer ein "c" enthält wurde die Transaktion storniert. | Nominal | object |
|StockCode | Produktnummer, 5-stellige Nummer, welche jedem einzelnem Produkt eindeutig zugeordnet ist. | Nominal | object |
|Description | Beschreibung des Produkts. | Nominal | object |
|Quantity | Menge der einzelnen Produkte pro Transaktion. | Verhaeltnis | int |
|InvoiceDate | Uhrzeit und Datum der Rechnung. | Intervall | date |
|UnitPrice | Preis pro Produkt. | Verhaeltnis | float |
|CustomerID | Kundennummer, 5-stellige Nummer, welche je einem Kunden eindeutig zugeordnet ist. | Nominal | object |
|Country | Name des Landes in welchem der Kunde ansässig ist. | Nominal | category |
|TotalPrice | Produkt aus Quantity und UnitPrice. | Verhaeltnis | float |

## Setup

In [3]:
# Setup
# Import relevanter Python-Bibliotheken

import pandas as pd
import altair as alt

## Data

## Import data

In [4]:
# Import des Datensatzes
raw_data = "https://raw.githubusercontent.com/ArditFazliu/Analytics-und-Data-Storytelling/main/data/raw_data.csv" 
df = pd.read_csv(raw_data, encoding= 'unicode_escape')

### Data structure

### Data corrections

## Analysis

### Descriptive statistics

### Exploratory data analysis

## Visualizations

### Visualization ideas

### Save Visualizations



Save your draft visualizations in the folder `reports/visualizations/`. Use a meaningful name (always include the word `draft` and a `timestamp`in your filename).

## Conclusion and recommended action

### Literature

Alvarez & Marsal. (2021). The shape of retail: the true cost of online. In https://www.alvarezandmarsal.com/.

Statista. (2022a, Mai 11). Prognose der Marktentwicklung des Online-Handels in Deutschland bis 2022. https://de.statista.com/statistik/daten/studie/202905/umfrage/prognostiziertes-marktvolumen-des-deutschen-versandhandels/

Statista. (2022b, Juli 25). Prognose der Umsätze im B2B-E-Commerce in den USA bis 2025. https://de.statista.com/statistik/daten/studie/1321716/umfrage/umsaetze-im-b2b-e-commerce-in-den-usa/